In [7]:
import pandas as pd
import os
import glob
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category = FutureWarning)
from tqdm import tqdm
import sys
import re

sys.path.insert(0, '../../../src/utils')
from import_data import *

smi_ke = pd.read_excel("~/Dropbox/Bolivia_Project/social-media-influencers-africa/data/02-randomize/KE/03-assignment/output/RandomizedTwitterSampleKE.xlsx")
smi_ke_1 = pd.read_excel("~/Dropbox/Bolivia_Project/social-media-influencers-af/data/02-randomize/KE/03-assignment/output/RandomizedTwitterSampleKE.xlsx")
smi_ke_2 = pd.read_excel("~/Dropbox/Bolivia_Project/social-media-influencers-af/data/02-randomize/KE/03-assignment/output/RandomizedTwitterSampleKE_batch2.xlsx")

smi_sa = pd.read_excel("~/Dropbox/Bolivia_Project/social-media-influencers-africa/data/02-randomize/SA/03-assignment/output/RandomizedTwitterSampleSA.xlsx")
smi_sa_1 = pd.read_excel("~/Dropbox/Bolivia_Project/social-media-influencers-af/data/02-randomize/SA/03-assignment/output/RandomizedTwitterSampleSA.xlsx")
smi_sa_2 = pd.read_excel("~/Dropbox/Bolivia_Project/social-media-influencers-af/data/02-randomize/SA/03-assignment/output/RandomizedTwitterSampleSA_batch2.xlsx")

smis = pd.concat([smi_ke,smi_ke_1,smi_ke_2,smi_sa,smi_sa_1,smi_sa_2], ignore_index=True)
smis = smis[smis['treatment']==1]
smis_list = list(smis.username)

def summ_followers2(df):
    if 'like_count' in df.columns:
        df = df.rename(columns={'like_count':'total_likes'})
    else:
        df = df.rename(columns={'public_metrics.like_count':'total_likes'}) 
    df['t_verifiability_smi'] = df['verifiability'] * df['interaction_smi']
    df['t_verifiability_ac'] = df['verifiability'] * df['interaction_ac']
    df['t_eng_smi'] = df['eng'] * df['interaction_smi']
    df['t_eng_ac'] = df['eng'] * df['interaction_ac']
    df['t_non_ver_smi'] = df['non_ver'] * df['interaction_smi']
    df['t_non_ver_ac'] = df['non_ver'] * df['interaction_ac']
    df['t_true_smi'] = df['true'] * df['interaction_smi']
    df['t_true_ac'] = df['true'] * df['interaction_ac']
    df['t_fake_smi'] = df['fake'] * df['interaction_smi']
    df['t_fake_ac'] = df['fake'] * df['interaction_ac']

    
    cols = (['interaction_smi', 'interaction_ac'] + 
            [col for col in df.columns if 't_' in col])
    df_agg = df[['author_id'] + 
        cols].groupby(['author_id']).sum().reset_index()
    
    return df_agg

####### Code:

for country in ['KE', 'SA']:
    for stage in ['stage1_2', 'stage3_4', 'stage5_6']: 
        base1 = pd.read_parquet(f'../../../data/04-analysis/{country}/baseline/baseline_batch1_smi_ac.parquet')

        df_final = get_endline_data(country, stage = stage, type_data = 'predicted', 
                                    base_path = '../../../')
        df_final['eng'] = np.where((df_final['lang'] == 'en'), 1, 0)
        df_final['total_shares'] = np.where(df_final['total_shares'] == 216736900298309634, 34, 
                                            df_final['total_shares'])
        df_final['total_reactions'] = np.where((df_final['total_reactions'] == 216736900298309635), 35, 
                               df_final['total_reactions'])
        df_final['total_reactions'] = np.where((df_final['total_reactions'] == 216736900298309634), 34, 
                               df_final['total_reactions'])

        df_final['fake'] = np.where((df_final['verifiability'] == 1) & (df_final['true'] == 0), 1, 0)
        df_final['fake'] = np.where((df_final['verifiability'] == 0), np.nan, df_final['fake'])
        df_final['non_ver'] = np.where((df_final['verifiability'] == 0) & (~df_final['verifiability'].isnull()),
                                   1, 0)
        # Extraer el nombre de usuario de los retweets
        df_final['reposted'] = df_final['text'].str.extract(r'^RT @(\w+):')
        # Completar los valores nulos con la columna handle
        df_final['reposted'] = df_final['reposted'].fillna(df_final['author_id'])
        df_final['interaction_ac'] = (df_final['reposted'] == 'AfricaCheck').astype(int)
        df_final['interaction_smi'] = (df_final['reposted'].isin(smis_list)).astype(int)
        df_final = df_final.drop(['reposted'], axis = 1)
        df_no_rt = df_final[(df_final['text'].str.contains('RT @', 
                    case=True, regex=False))].reset_index(drop=True)

        df_no_rt_agg = summ_followers2(df_no_rt).reset_index(drop=True)
        
        df_no_rt_agg.rename(columns = {'author_id':'follower_id'}, inplace = True)

        base1 = base1.merge(df_no_rt_agg, on=['follower_id'], 
                            how='left')

        cols_base = (['interaction_smi', 'interaction_ac'] + 
            [col for col in base1.columns if 't_' in col])

        for x in cols_base:
            base1[x] = np.where(base1[x].isnull(), 0, base1[x])
    
        base1.to_parquet(f'../../../data/04-analysis/{country}/{stage}/verifiability_batch1_smi_ac.parquet')

100%|███████████████████████████████████████████| 25/25 [00:58<00:00,  2.32s/it]


In [8]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6410374 entries, 0 to 6410373
Data columns (total 21 columns):
 #   Column            Dtype  
---  ------            -----  
 0   author_id         object 
 1   id                object 
 2   conversation_id   object 
 3   created_at        object 
 4   text              object 
 5   lang              object 
 6   total_shares      int32  
 7   reply_count       int32  
 8   like_count        int32  
 9   quote_count       int32  
 10  impression_count  int32  
 11  lang2             object 
 12  total_reactions   int64  
 13  total_comments    int64  
 14  verifiability     float64
 15  true              float64
 16  eng               int64  
 17  fake              float64
 18  non_ver           int64  
 19  interaction_ac    int64  
 20  interaction_smi   int64  
dtypes: float64(3), int32(5), int64(6), object(7)
memory usage: 904.8+ MB
